In [ ]:
!pip uninstall -y oao24
!pip install git+https://github.com/ArcetriAdaptiveOptics/OAO24.git

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
from oao24.package_data import InfraredExampleData, VisibleExampleData
from oao24 import image_processing

In [ ]:
background_image = InfraredExampleData.get_camera_dark_data()
cl_raw_image_cube = InfraredExampleData.get_close_loop_data_cube()
ao_image = image_processing.make_master_image(cl_raw_image_cube, background_image)

In [ ]:

RAD2ARCSEC=180/np.pi*3600
pupil_diameter = 1.52 
wavelength = 1.65e-6 # 1310e-9
# Plate scale is typically saved in the FITS image header
# or it can be calibrated using 2 sources of known distance
# Here we use the Papyrus nominal optical design values
# F/# = 23
# The DL PSF size in m on the focal is F/H * wavelength
cred3_pixel_in_meter = 15e-6
f_number =  23
telescope_focal_length = f_number * pupil_diameter
# so the size of the pixel in arcsec is 
pixel_scale_in_arcsec =cred3_pixel_in_meter / telescope_focal_length * RAD2ARCSEC 

# the DL size (=lambda/D) in units of arcsec or pixels are therefore
dl_size_in_arcsec = wavelength / pupil_diameter * RAD2ARCSEC 
dl_size_in_pixels = dl_size_in_arcsec / pixel_scale_in_arcsec

print("C-Red3 pixel scale: %g arcsec/pixel" % pixel_scale_in_arcsec)
print("DL PSF size: %g arcsec" % dl_size_in_arcsec)
print("DL PSF size: %g pixels" % dl_size_in_pixels)

star_roi = ao_image[240:296, 335:391]
star_roi_cut_index = (29, slice(15, 45))

In [ ]:

#SR estimation assuming a gaussaian model
amp = star_roi.max()
x0 = np.where(star_roi == star_roi.max())[1][0]
y0 = np.where(star_roi == star_roi.max())[0][0]
sigma_x = 2.5/(2*np.sqrt(2*np.log(2)))
sigma_y = sigma_x
model_gauss = models.Gaussian2D(amp, x0, y0, sigma_x, sigma_y)
fitter = fitting.LevMarLSQFitter()
roi_shape = star_roi.shape
y, x = np.mgrid[:roi_shape[0], :roi_shape[1]]
best_fit_gauss = fitter(model_gauss, x, y, z = star_roi)
psf_residual_gauss= best_fit_gauss(x,y)-star_roi


In [11]:

Ntot = star_roi.sum()

alpha = np.pi/(f_number*wavelength)*15e-6
amp_dl  = 3 *alpha* Ntot / 32
sr = best_fit_gauss.parameters[0]/amp_dl
#computing SR from fft of pupil


In [12]:

from oao24.mask import AnnularMask

Npix = 200
pupil = np.zeros((Npix,Npix))
pupil_radius_in_pix = 76
pupil_center = (99,99)
obs_radius_in_pix = int(np.round(0.33*pupil_radius_in_pix))

pupil_mask_obj = AnnularMask(
    frameShape=pupil.shape,
    maskRadius=pupil_radius_in_pix,
    maskCenter=pupil_center,
    inRadius=obs_radius_in_pix)

pupil_mask = pupil_mask_obj.asTransmissionValue()

phase = np.ma.array(pupil, mask = pupil_mask)

#computing transmitted electric field
Ut = 1 * np.exp(1j * phase)

Ut.fill_value = 0
Ut.data[Ut.mask == True] = 0

#padding transmitted electric field
Npad = 5
padded_frame_size = int(np.round(np.max(phase.shape) * Npad))
padded_Ut = np.zeros((padded_frame_size, padded_frame_size), dtype=complex)
padded_Ut[0 : Ut.shape[0], 0 : Ut.shape[1]] = Ut   

#computing angular frequencies
dxi = 0.5*Dtel/pupil_radius_in_pix
deta = dxi 

#in rad
x = np.fft.fftshift(np.fft.fftfreq(padded_Ut.shape[1], dxi)) * wl 
y = np.fft.fftshift(np.fft.fftfreq(padded_Ut.shape[0], deta)) * wl


#computing psf
dl_psf = np.abs(np.fft.fftshift(np.fft.fft2(padded_Ut)))**2

total_dl_flux = dl_psf.sum()
total_meas_flux = star_roi.sum()

dl_psf_norm = dl_psf * total_meas_flux/total_dl_flux

from scipy.ndimage import zoom

pix_scale_dl_in_rad = x[-1]-x[-2]
pix_scale_meas_in_rad = (wl/Dtel)/3.6

scale_factor = pix_scale_meas_in_rad/pix_scale_dl_in_rad
measured_shape = star_roi.shape

dl_shape = dl_psf_norm.shape

zoom_factors = (measured_shape[0] / dl_shape[0] *scale_factor,
                measured_shape[1] / dl_shape[1] *scale_factor)
dl_psf_rebinned = zoom(dl_psf_norm, zoom_factors)



#assert dl_psf_rebinned.shape == star_roi.shape, "Rebinning failed, shapes do not match"

# def rebin(arr, new_shape):
#     """
#     Rebin the array `arr` into a new shape `new_shape`.
#     This function works by averaging blocks of pixels.
#     """
#     # Compute the shape of the input array
#     old_shape = arr.shape
#
#     if old_shape[0] % new_shape[0] != 0 or old_shape[1] % new_shape[1] != 0:
#         raise ValueError("Cannot rebin: dimensions are not compatible.")
#     # Calculate the shape of the re-binned array
#     sh = (old_shape[0] // new_shape[0], new_shape[0],
#           old_shape[1] // new_shape[1], new_shape[1])
#     # Use reshaping and averaging
#     return arr.reshape(sh).mean(axis=(1, 3))
#
# # Rebin the normalized DL PSF to match star_roi shape
# dl_psf_rebinned = rebin(dl_psf_norm, star_roi.shape)


sr = best_fit_gauss.parameters[0]/dl_psf_rebinned.max()
return sr, phase, dl_psf_norm, x, dl_psf_rebinned, star_roi

ModuleNotFoundError: No module named 'arte'